# Step 1: Pre-processing model and reanalsyis data

---

## Instructions for activating the Jupyter kernel for the `cmip6hack-multigen` conda environment

In a Jupyterlab terminal, navigate to the `/cmip6hack-multigen/` folder and run the command:
```bash
source spinup_env.sh
```
which will create the `cmip6hack-multigen` conda environment and install it as a python kernel for jupyter.

Then, switch the kernel (drop-down menu in the top right hand corner) to cmip6hack-multigen and restart the notebook.

### Pre-process climate model output in GCS

This notebook uses [`intake-esm`](https://intake-esm.readthedocs.io/en/latest/) to ingest and organize climate model output from various model generations and resave their time-mean fields locally.

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import xskillscore as xs
import xesmf as xe
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util
import preprocess as pp
import qc

In [23]:
varnames = ['tas', 'pr', 'psl']
timeslice = slice('1981', '2010')
coarsen_size = 2

In [24]:
col_dict = pp.get_ipcc_collection()

In [ ]:
ds_dict = {}
for mip_id, col in tqdm(col_dict.items()):
    ds_dict[mip_id] = {}
    for varname in varnames:
        print("Loaded: variable_id `", varname, "` from activity_id `",mip_id,"`")
        cat = col.search(
            experiment_id='historical',
            variable_id=varname,
            table_id='Amon'
        )

        if cat.df.size == 0: continue

        with util.HiddenPrints():
            dset_dict = cat.to_dataset_dict(
                aggregate=False,
                zarr_kwargs={'consolidated': True, 'decode_times': False}
            )

        ds_dict[mip_id][varname] = {}
        for key, ds in dset_dict.items():
            # rename spatial dimensions if necessary
            if ('longitude' in ds.dims) and ('latitude' in ds.dims):
                ds = ds.rename({'longitude':'lon', 'latitude': 'lat'})

            # Need this temporarily because setting 'decode_times': True is broken
            ds = xr.decode_cf(ds)
            ds['time'] = ds['time'].astype('<M8[ns]')
            ds['time'].values = np.array(
                pd.to_datetime(
                    util.vec_dt_replace(pd.Series(ds['time'].values), day=1.)
                )
            )

            repeats = len(ds['time']) - len(np.unique(ds['time']))
            if repeats != 0:
                print(f"Skip {key} before datetime conflict.")
                continue

            ds = ds.squeeze() # get rid of member_id (for now)

            chunks = {'lat':ds['lat'].size, 'lon':ds['lon'].size, 'time':30}
            ds = ds.chunk(chunks)

            if timeslice is not None:
                ds = ds.sel(time=timeslice)

            with util.HiddenPrints():
                try:
                    ds_new = util.regrid_to_common(ds[varname])
                except:
                    print(f"Skip {key} due to regridding conflict.")
                    continue

            ds_new.attrs.update(ds.attrs)
            ds_new = qc.quality_control(ds_new, varname, key, mip_id)

            ds_new.attrs['name'] = "-".join(key.split(".")[1:3])

            for coord in ds_new.coords:
                if coord not in ['lat', 'lon', 'time']:
                    ds_new = ds_new.drop(coord)

            member_id = key.split(".")[4]
            ds_new = ds_new.expand_dims(
                {'ensemble': np.array([ds_new.attrs['name'] + "-" + member_id])}, 0
            )
            
            ds_new = ds_new.assign_coords({
                'member_id': member_id,
                'source_id': key.split(".")[2],
                'mip_id': key.split(".")[0]
            })

            ds_new.attrs['mip_id'] = mip_id

            coarsen_dict = {'lat': coarsen_size, 'lon': coarsen_size}
            ds_new = ds_new.coarsen(coarsen_dict, boundary='exact').mean()

            ds_dict[mip_id][varname][key] = ds_new  # add this to the dictionary

Loaded: variable_id ` tas ` from activity_id ` far `
Loaded: variable_id ` pr ` from activity_id ` far `
Loaded: variable_id ` psl ` from activity_id ` far `
Loaded: variable_id ` tas ` from activity_id ` sar `
Loaded: variable_id ` pr ` from activity_id ` sar `
Loaded: variable_id ` psl ` from activity_id ` sar `
Loaded: variable_id ` tas ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.r1i1p1f1.Amon.tas.gn before datetime conflict.
Loaded: variable_id ` pr ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.r1i1p1f1.Amon.pr.gn before datetime conflict.
Loaded: variable_id ` psl ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.r1i1p1f1.Amon.psl.gn before datetime conflict.
Loaded: variable_id ` tas ` from activity_id ` cmip3 `
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.r1i1p1f1.Amon.tas.gn before datetime conflict.
Loaded: variable_id ` pr ` from activity_id ` cmip3 `
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.r1i1p1f1.Amon.pr.gn before datetime confli

In [15]:
ds_dict['cmip6']['tas']['CMIP.BCC.BCC-CSM2-MR.historical.r1i1p1f1.Amon.tas.gn']

<xarray.DataArray (ensemble: 1, time: 360, lat: 90, lon: 180)>
dask.array<mean_agg-aggregate, shape=(1, 360, 90, 180), dtype=float64, chunksize=(1, 30, 90, 180), chunktype=numpy.ndarray>
Coordinates:
  * ensemble  (ensemble) <U15 'BCC-BCC-CSM2-MR'
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
  * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Attributes:
    regrid_method:          bilinear
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          Standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  2289.0
    cmor_version:           3.3.2
    comment:                The model integration starts from the piControl e...
    contact:                Dr. Tongwen Wu(twwu@cma.gov.cn)
    creation_date:          2018-11-26T05:08:26Z
    data_specs_version:     01.00.27
    description:            DECK: historical
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    external_variables:     areacella
    forcing_index:          1
    frequency:              mon
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-CSM2...
    grid:                   T106
    grid_label:             gn
    history:                2018-11-26T05:08:26Z ; CMOR rewrote data to be co...
    initialization_index:   1
    institution:            Beijing Climate Center, Beijing 100081, China
    institution_id:         BCC
    license:                CMIP6 model data produced by BCC is licensed unde...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_source_id:       BCC-CSM2-MR
    parent_time_units:      days since 1850-01-01
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    references:             Model described by Tongwen Wu et al. (JGR 2013; J...
    run_variant:            forcing: greenhouse gases,solar constant,aerosol,...
    source:                 BCC-CSM 2 MR (2017):   aerosol: none  atmos: BCC_...
    source_id:              BCC-CSM2-MR
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               Amon
    table_info:             Creation Date:(30 July 2018) MD5:e53ff52009d0b97d...
    title:                  BCC-CSM2-MR output prepared for CMIP6
    tracking_id:            hdl:21.14100/007da5e5-12bb-4fe9-8df9-5cd473280650
    variable_id:            tas
    variant_label:          r1i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu
    intake_esm_varname:     None
    name:                   BCC-BCC-CSM2-MR
    mip_id:                 cmip6

In [4]:
ens_dict = pp.load_ensembles(varnames, timeslice=timeslice)

Loaded: variable_id ` tas ` from activity_id ` far `
Loaded: variable_id ` pr ` from activity_id ` far `
Loaded: variable_id ` psl ` from activity_id ` far `
Loaded: variable_id ` tas ` from activity_id ` sar `
Loaded: variable_id ` pr ` from activity_id ` sar `
Loaded: variable_id ` psl ` from activity_id ` sar `
Loaded: variable_id ` tas ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` pr ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` psl ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` tas ` from activity_id ` cmip3 `
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` pr ` from activity_id ` cmip3 `
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` psl ` from activity_id ` cmip3 `
Load

/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,


#### Extracting time-mean

In [5]:
ens_dict = util.dict_func(ens_dict, xr.Dataset.mean, on_self=True, dim =['time'], keep_attrs=True)

In [6]:
ens_dict = util.dict_func(ens_dict, xr.Dataset.compute, on_self=True)

/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


#### Adding ensemble-mean to the ensemble

In [7]:
ens_dict = util.add_ens_mean(ens_dict)

### Pre-process observational data products

In [8]:
era5 = pp.load_era("../data/raw/reanalysis/ERA5_mon_2d.nc", timeslice=timeslice, coarsen_size=2)

### Save interim files

In [12]:
interim_path = "../data/interim/"
era5.to_zarr(interim_path + "era5_timemean", "w")

In [13]:
for key, ens in ens_dict.items():
    ens.to_zarr(interim_path + f"{key}_timemean", "w")